In [1]:
import tqdm
import os
import re
import codecs
import jieba
import pickle
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient

In [2]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [3]:
es_idx = IndicesClient(es)

In [236]:
a = os.popen('''
curl -X PUT 'localhost:9200/main_idx' -d '
{
  "mappings": {
    "docs": {
      "properties": {
        "pure_text": {
          "type": "text",
          "analyzer": "ik_max_word",
          "search_analyzer": "ik_max_word"
        },
        "processed_text": {
          "type": "text",
          "analyzer": "ik_max_word",
          "search_analyzer": "ik_max_word"
        }
      }
    }
  }
}'
''').read()
for line in a.split('\n'):
    print(line)

{"acknowledged":true,"shards_acknowledged":true,"index":"main_idx"}


In [35]:
# res1 = os.popen('''
# curl -X PUT 'localhost:9200/main_idx/docs/1' -d '
# {
#     "pure_text": "中华人民共和国国歌",
#     "processed_text": "中华 人民 共和国 国歌"
# }'
# ''').read()
# for line in res1.split('\n'):
#     print(line)

{"_index":"main_idx","_type":"docs","_id":"1","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"created":true}


In [39]:
# res = os.popen('''
# curl 'localhost:9200/main_idx/docs/_search'  -d '
# {
#   "query" : { "match" : { "pure_text" : "中华人民共和国" }}
# }'
# ''').read()
# for line in res.split('\n'):
#     print(line)

{"took":1,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":1,"max_score":0.25811607,"hits":[{"_index":"main_idx","_type":"docs","_id":"1","_score":0.25811607,"_source":
{
    "pure_text": "中华人民共和国国歌",
    "processed_text": "中华 人民 共和国 国歌"
}}]}}


In [70]:
# es_idx.exists(index='text_search')

False

In [235]:
es_idx.delete(index='_all')

{'acknowledged': True}

In [237]:
# all_question = pd.read_csv('./data/description.csv')
Herceptin_question = pd.read_excel('./data/Herceptin.xlsx')
# all_question = pd.read_excel('./data/Herceptin_funding_filtered.xlsx')
# all_question = pd.read_excel('./data/Herceptin_patient_filtered.xlsx')
Kadcyla_question = pd.read_excel('./data/Kadcyla.xlsx')
Perjeta_1_question = pd.read_excel('./data/Perjeta_1.xlsx')
Perjeta_2_question = pd.read_excel('./data/Perjeta_2.xlsx')
Tecentriq_question = pd.read_excel('./data/Tecentriq.xlsx')
Xeloda_question = pd.read_excel('./data/Xeloda.xlsx')
No_product_question = pd.read_excel('./data/No_product_merged.xlsx')

all_question = pd.concat([Herceptin_question, Kadcyla_question, Perjeta_1_question, Perjeta_2_question, Tecentriq_question, Xeloda_question, No_product_question])
print('all_question: ', len(all_question))
all_question.head(5)

all_question.dropna(axis=0, inplace=True, subset=['Description'])
all_question.isna().sum()

print('all_question: ', len(all_question))

jieba.load_userdict('./dict data/chinese_word_dict.txt')
chinese_word_dict = pickle.load(open('./dict data/chinese_word_dict.pickle', 'rb'))
stopwords = [line.strip() for line in codecs.open('./stopwords-zh.txt', 'r', encoding='utf8').readlines()]

def keyword_extract(text, word_dict):
#     text = list(jieba.cut(text, cut_all=False))
    text = list(jieba.cut_for_search(text))
    union_text = list(set(text).intersection(set(word_dict)))
    return " ".join(union_text)

all_question['KeyWords'] = all_question.Description.apply(lambda x: keyword_extract(x, chinese_word_dict))

# 去除所有“[]”内的文本
def remove_special_string(text):
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'（[^）]*\）', '', text)
    text = re.sub(r'\<[^>]*\>', '', text)
#     text = re.sub(r'[0-9]*', '', text)
#     text = re.sub('患者', '', text)
#     text = re.sub('家属', '', text)
#     text = re.sub('咨询', '', text)
#     text = re.sub('询问', '', text)
    return text
def remove_stopwords(text, stopwords):
    text_cutted = jieba.cut(text, cut_all=True)
    text_removed = []
    for word in text_cutted:
        if word in stopwords or word.strip()=="":
            continue
        text_removed.append(word)
    return "  ".join(text_removed)

all_question['Description'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Processed.apply(lambda x: remove_stopwords(x, stopwords))
all_question.head()

all_question:  1970
all_question:  1656


,Case Number,Description,KeyWords,Processed
0,1420656.0,赫赛汀 配了之后结冰了， 能否继续使用。,赫赛汀 使用,赫赛汀 配 之后 结冰 继续 使用
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用,无 赫赛汀 使用 患者,赫赛汀 粉末 形状 保存 赫赛汀 稀释 稀释液 出现 结冰 冰情 情况 ...
2,1434184.0,请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献,查 血栓 无 赫赛汀 栓塞 血栓栓塞 文献,帮忙 查 赫赛汀 血栓 血栓栓塞 栓塞 方面 数据 和文 文献
3,1436011.0,拿到的赫赛汀呈现块状是否正常,赫赛汀 正常,拿到 赫赛汀 呈现 块状 是否 正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用,下 使用 现 赫赛汀 配置 溶液,常规 配置 保存 赫赛汀 复 溶 溶液 保存 存在 2 8 度 情况...


In [259]:
all_query_text = []
for idx, row in all_question.iterrows():
    all_query_text.append(str(row['Description']) + "\n\n")
    
# all_query_text_f = open('all_query_text.txt', 'w')
# all_query_text_f.writelines(all_query_text)
# all_query_text_f.close()

In [239]:
doc_list = []
for idx,row in all_question.iterrows():
    doc = {"pure_text": row['Description'], "processed_text": row['KeyWords']}
    doc_list.append(doc)
print(len(doc_list))
print(doc_list[0])

1656
{'pure_text': '赫赛汀  配了之后结冰了， 能否继续使用。', 'processed_text': '赫赛汀 使用'}


In [218]:
res1 = os.popen('''
curl -X POST 'localhost:9200/main_idx/docs' -d '
{
    "pure_text": "您好，请帮忙查一下数据，谢谢！需要绝经后女性乳腺癌的以下各项指标的比例。ER阳性，PR阳性，AR，HER2阳性，LuminalA，LuminalB，三阴性。感谢，盼复。",
    "processed_text": "您好 ， 请 帮忙 查 一下 数据 ，谢谢 ！ 需要 绝经 后 女性 乳腺癌 的 以下 各项 指标 的 比例 。ER 阳性 ， PR 阳性 ， AR，HER2 阳性 ， LuminalA ， LuminalB ， 三阴性 。 感谢 ， 盼复 。 "
}'
''').read()
for line in res1.split('\n'):
    print(line)

{"_index":"main_idx","_type":"docs","_id":"AWsliWRfFGUTYcujgzY1","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"created":true}


In [ ]:
for doc in doc_list:
    temp = os.popen('''
    curl -X POST 'localhost:9200/main_idx/docs' -d '
    {}
    '
    '''.format(str(doc).replace("\"", "\'").replace("\'", "\""))).read()
    for line in temp.split('\n'):
        print(line)
        
#     print('''
#     curl -X PUT 'localhost:9200/ik_test/docs/{}' -d '
#     {}
#     '
#     '''.format(str(idx), str(doc_list[idx]).replace("\"", "\'").replace("\'", "\"")))
    
# curl -X PUT 'localhost:9200/ik_test/docs/1' -d '
# {
#     "content": "中华人民共和国国歌"
# }'

In [ ]:
res = os.popen('''
curl 'localhost:9200/main_idx/docs/_search?pretty=true'  -d '
{
  "query" : { "match" : { "processed_text" : "关于赫赛汀心脏毒性：1、临床 2、临床前" }}
}'
''').read()
for line in res.split('\n'):
    print(line)

In [96]:
for idx in range(len(doc_list)):
    es.index(index='text_search', id=idx, body=doc_list[idx])

In [250]:
query = "烦请帮忙查找以下方面的文献。1.三阴乳腺癌对希罗达耐药，应该用哪些药？2.三阴乳腺癌对其他药物耐药，对希罗达有效。"

In [260]:
# processed_text  pure_text
results = es.search(index='main_idx', body=
{
  "query" : { "match" : { "pure_text" : query }}
}
# {
#   "query": {
#     "bool": {
#       "must":     { "match": { "processed_text": query }},
#       "should":   { "match": { "pure_text": "帕妥珠单抗" }}
#     }
#   }
# }
,size=50)
hits = results['hits']['hits']
# results
for idx, res in enumerate(hits):
#     if "帕妥珠单抗" in res['_source']['pure_text']:
    print(str(idx+1) + ":  " + res['_source']['pure_text'])

1:   烦请帮忙查找以下方面的文献。1.三阴乳腺癌对希罗达耐药，应该用哪些药？2.三阴乳腺癌对其他药物耐药，对希罗达有效。
2:   烦请帮忙查找以下方面的文献  3.乳腺癌耐药与外泌体的关系。
3:   希罗达在三阴乳腺癌新辅助后强化治疗的治疗方面的文献
4:   请帮忙查询Clinical trial上所有注册的关于希罗达在三阴乳腺癌方面的研究的内容
5:   烦请帮忙查找以下方面的文献，4.希罗达对乳腺癌干细胞是否有作用。
6:   希罗达在乳腺癌方面的最新文献
7:   希罗达用于三阴乳腺癌的研究，最好是中文，英文也可
8:  Her-2阳性乳腺癌含希罗达的方案文献  er-2阴性乳腺癌含希罗达的方案文献-
9:   希罗达在三阴性乳腺癌中的文献和研究
10:   希罗达在三阴乳腺癌术后节拍化疗的相关PPT或者文献。谢谢！
11:   乳腺癌 希罗达 三阴型乳腺癌治疗
12:  乳腺癌希罗达的相关研究以及乳腺癌方面最新的文献材料
13:   乳腺癌治疗-希罗达-客户的问题为：1、目前希罗达在乳腺癌的定位是蒽环治疗失败或蒽环和紫杉均耐药才推荐使用，那么是否有希罗达与紫杉直接对照治疗乳腺癌的证据吗？希罗达是否可以代替紫杉作为一线之间使用？ 2、客户想阅读一下希罗达在晚期乳腺癌的实验原文，请帮忙收集一下哈
14:   1.近3年来希罗达在转移性乳腺癌方面的论文，中英文，全部
15:   乳腺癌希罗达的相关研究以及乳腺癌方面最新的文献材料
16:   希罗达联合拉帕替尼在乳腺癌治疗方面文献   
17:   您好，请帮忙查找希罗达在乳腺癌的最新进展的中文文献，以及乳腺癌维持治疗的最新数据，谢谢
18:   希罗达在三阴性乳腺癌化疗的课件
19:  问题描述： 希罗达在三阴乳腺癌的具体应用是否要跟紫衫或者铂类联用，联用一年？具体介入时间什么时候？
20:   想要希罗达乳腺方面的研究文献；
21:   乳腺癌三阳患者服用希罗达，要指南指证
22:   希罗达在乳腺癌XTvsAT方案试验原文
23:   客户需要近5年希罗达治疗乳腺癌的最新文献资料，
24:  想了解一下有关希罗达在乳腺癌的实验有哪些具体的文献
25:  希罗达晚期乳腺癌应用的相关文献研究
26:  患者乳腺癌,之前使用的药物耐药,目前使用希罗达单药治疗,每天需要4500毫克/升,但是希罗达500毫克

In [258]:
tokens_res = es_idx.analyze(index='main_idx', body={
    "analyzer": "ik_max_word",
#     "analyzer": "ik_smart",
    "text" : "'烦请  帮忙  查找  以下  下方  方面  文献  1  三阴乳腺癌  乳腺  乳腺癌  腺癌  希罗  希罗达  耐药  应该  药  2  三阴乳腺癌  乳腺  乳腺癌  腺癌  药物  耐药  希罗  希罗达  有效'"
})
for token in tokens_res['tokens']:
    print(token['token'])

烦请
帮忙
查找
查
找
以下
下方
方
方面
方
面
文献
1
三阴乳腺癌
阴
乳腺癌
乳腺
乳
腺癌
癌
乳腺
乳
腺
乳腺癌
乳腺
乳
腺癌
癌
腺癌
癌
希
罗
希罗达
罗
达
耐药
应该
应
药
2
三阴乳腺癌
阴
乳腺癌
乳腺
乳
腺癌
癌
乳腺
乳
腺
乳腺癌
乳腺
乳
腺癌
癌
腺癌
癌
药物
物
耐药
希
罗
希罗达
罗
达
有效
